<a href="https://colab.research.google.com/github/TigistW/Deep-Learning-Labs/blob/main/DL_Lab3_Ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import torch

In [83]:
torch.manual_seed(12)

In [84]:
class DenseLayer:
    def __init__(self, n_features, n_neurons):
        self.weights = 0.01 * torch.rand(n_neurons, n_features)
        self.biases = torch.zeros(1, n_neurons)
        self.output = None

    def forward(self, inputs):
        self.output = torch.matmul(inputs, self.weights.T) + self.biases

class Activation_ReLU:
    def forward(self, inputs):
        self.output = torch.max(torch.tensor(0.0), inputs)

class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + torch.exp(-inputs))

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = torch.exp(inputs) - torch.max(inputs, axis=1, keepdims=True).values
        summed_exp = torch.sum(exp_values, axis=1, keepdims=True)
        self.output = exp_values / summed_exp

class Loss_CategoricalCrossentropy:

    def __init__(self, y_pred, y_true):

      self.loss = torch.zeros(y_pred.shape[1])
      self.loss_mean = 0

      if y_pred.shape == y_true.shape:
        self.forward_with_hot_encoding(y_pred, y_true)
      elif y_pred.shape[0] == y_true.shape[0]:

        self.forward_without_encoding(y_pred, y_true)
      else:
        pass

    def forward_without_encoding(self, y_pred, y_true):
        for i,j in zip(y_pred,y_true):
          self.loss += (-(j*torch.log(i)))
        self.loss_mean = torch.sum(self.loss) /len(self.loss)

    def forward_with_hot_encoding(self, y_pred, y_true):
        self.loss = -torch.sum(y_true * torch.log(y_pred))

class NeuralNetwork:
    def __init__(self, input, num_features, num_neurons, activation_function):

        self.num_features = num_features
        self.activation = activation_function
        self.num_neurons = num_neurons
        self.input = input
        self.output = None

    def forward(self):
        self.layer = DenseLayer(self.num_features, self.num_neurons)
        self.layer.forward(self.input)
        self.activation.forward(self.layer.output)
        self.output = self.activation.output
class Accuracy:
  def acc_percentage(self, predicted, target):
      correct_predictions = (predicted == target).sum().item()
      total_samples = len(target)
      self.accuracy = correct_predictions / total_samples * 100.0



In [85]:
# given parameters
num_feat_one = 4
num_feat_two = 4
n_layer = 4
output_layer_neurons = 3
neuron_in_every_layers = [18,18,18,3]
model_one_activation_functions = [Activation_ReLU(), Activation_ReLU(), Activation_ReLU(), Activation_Softmax()]
model_two_activation_functions = [Activation_Sigmoid(), Activation_Sigmoid(), Activation_Sigmoid(), Activation_Softmax()]
n_sample = 4

input_one = torch.rand(n_sample, num_feat_one)
input_two = torch.rand(n_sample, num_feat_two)

# input_one = (100000 - 0) * torch.rand(n_sample, num_feat_one) + 0
# input_two = (100000 - 0) * torch.rand(n_sample, num_feat_two) + 0

for i in range(n_layer):
  relu_model = NeuralNetwork(input_one, num_feat_one,neuron_in_every_layers[i], model_one_activation_functions[i])
  relu_model.forward()
  num_feat_one = relu_model.output.shape[1]
  input_one = relu_model.output

  sig_model = NeuralNetwork(input_two, num_feat_two,neuron_in_every_layers[i], model_two_activation_functions[i])
  sig_model.forward()
  num_feat_two = sig_model.output.shape[1]
  input_two = sig_model.output


In [86]:
#generate target values
y_pred_model_one = relu_model.output

y_true_model_one = torch.randint(num_feat_one, (n_sample,))
y_true_model_one_encoded = torch.eye(y_pred_model_one.shape[1])[torch.randint(y_pred_model_one.shape[1], size=(n_sample,))]
log_loss_one = Loss_CategoricalCrossentropy(y_pred_model_one, y_true_model_one)
log_loss_one_encoded = Loss_CategoricalCrossentropy(y_pred_model_one, y_true_model_one_encoded)

y_pred_model_two = sig_model.output

y_true_model_two = torch.randint(num_feat_one, (n_sample,))
y_true_model_two_encoded = torch.eye(y_pred_model_two.shape[1])[torch.randint(y_pred_model_two.shape[1], size=(n_sample,))]
log_loss_two = Loss_CategoricalCrossentropy(y_pred_model_two, y_true_model_two)
log_loss_two_encoded = Loss_CategoricalCrossentropy(y_pred_model_two, y_true_model_two_encoded)

log_loss_one.loss_mean,log_loss_one_encoded.loss, log_loss_two.loss_mean,log_loss_two_encoded.loss,


(tensor(4.3944), tensor(4.3944), tensor(3.2959), tensor(4.4075))

In [87]:
accuracy_relu = Accuracy()
accuracy_sig = Accuracy()

target_value = torch.randint(num_feat_one, (n_sample,))

pred_index_relu = torch.argmax(relu_model.output,axis=1)
pred_index_sig = torch.argmax(sig_model.output,axis=1)

accuracy_relu.acc_percentage(pred_index_relu, target_value)
accuracy_sig.acc_percentage(pred_index_sig, target_value)

accuracy_relu.acc_percentage(pred_index_relu, target_value)
accuracy_sig.acc_percentage(pred_index_sig, target_value)

accuracy_relu.accuracy, accuracy_sig.accuracy

(0.0, 25.0)